In [ ]:
import sys
import os.path as op

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse

sys.path.append("/Users/dschonhaut/code/style")
from colors import get_colors
co, palettes = get_colors()

sys.path.append("/Users/dschonhaut/code")
from general.array import array_operations as aop
from general.basic.helper_funcs import *
from general.basic.str_methods import *
import general.nifti.nifti_ops as nops
import general.nifti.nifti_plotting as niiplot
colors = [
    "2E45B8",
    "3EBCD2",
    "FF4983",
    "1DC9A4",
    "F9C31F",
    "B38FE7",
    "F97A1F",
    "E3120B",
]
font = {"tick": 10, "label": 12, "title": 14, "annot": 12}
pad = {"tick": 2, "label": 5, "title": 8}
colws = {1: 2.05, 2: 3.125, 3: 7.28346}
lws = {"axis": 0.8, "line": 1.5, "marker": 0.8}

%matplotlib inline
mpl.rcParams["axes.axisbelow"] = True
mpl.rcParams["axes.formatter.offset_threshold"] = 2
mpl.rcParams["axes.grid"] = True
mpl.rcParams["axes.grid.which"] = "both"
mpl.rcParams["axes.labelpad"] = pad["label"]
mpl.rcParams["axes.labelsize"] = font["label"]
mpl.rcParams["axes.linewidth"] = lws["axis"]
mpl.rcParams["axes.prop_cycle"] = mpl.cycler("color", colors)
mpl.rcParams["axes.spines.right"] = False
mpl.rcParams["axes.spines.top"] = False
mpl.rcParams["axes.titlepad"] = pad["title"]
mpl.rcParams["axes.titlesize"] = font["title"]
mpl.rcParams["figure.autolayout"] = True
mpl.rcParams["figure.dpi"] = 300
mpl.rcParams["figure.figsize"] = (colws[3], colws[3]*0.618034)
mpl.rcParams["figure.labelsize"] = font["label"]
mpl.rcParams["figure.subplot.hspace"] = 0.2
mpl.rcParams["figure.subplot.wspace"] = 0.2
mpl.rcParams["figure.titlesize"] = font["title"]
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.serif'] = "Times New Roman"
mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams["grid.alpha"] = 0.5
mpl.rcParams["grid.color"] = "#B7C6CF"
mpl.rcParams["grid.linewidth"] = 0.2
mpl.rcParams["hist.bins"] = 30
mpl.rcParams["legend.borderaxespad"] = 0
mpl.rcParams["legend.fontsize"] = font["tick"]
mpl.rcParams["legend.frameon"] = False
mpl.rcParams["legend.handletextpad"] = 0.4
mpl.rcParams["legend.markerscale"] = 1
mpl.rcParams["legend.title_fontsize"] = font["label"]
mpl.rcParams["lines.color"] = colors[0]
mpl.rcParams["lines.linewidth"] = lws["line"]
mpl.rcParams["lines.markeredgewidth"]: lws["marker"]
mpl.rcParams["lines.markersize"] = 8
mpl.rcParams["patch.facecolor"] = colors[1]
mpl.rcParams["patch.linewidth"] = lws["marker"]
mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["savefig.bbox"] = "tight"
mpl.rcParams["savefig.directory"] = "~/Downloads"
mpl.rcParams["savefig.format"] = "pdf"
mpl.rcParams["savefig.pad_inches"] = 0.05
mpl.rcParams["xtick.labelsize"] = font["tick"]
mpl.rcParams["xtick.major.size"] = 4
mpl.rcParams["xtick.major.width"] = lws["axis"]
mpl.rcParams["xtick.minor.ndivs"] = 2
mpl.rcParams["ytick.labelsize"] = font["tick"]
mpl.rcParams["ytick.major.size"] = 4
mpl.rcParams["ytick.major.width"] = lws["axis"]
mpl.rcParams["ytick.minor.ndivs"] = 2


In [ ]:
def tukey_outlier(vec, iqr_mult=1.5, which="upper"):
    """Return the outlier threshold using Turkey's Method.

    Upper bound = Q3 + (iqr_mult * IQR)
    Lower bound = Q1 - (iqr_mult * IQR)

    where iqr_mult is a constant that is typically 1.5.

    Parameters
    ----------
    vec : array-like
        Vector of values.
    which : str
        Which threshold to return. Options are "upper", "lower", or
        "both".

    Returns
    -------
    thresh : float or tuple
        Outlier threshold(s).
    """
    q1, q3 = np.percentile(vec, (25, 75))
    iqr = q3 - q1
    lower_thresh = q1 - (iqr_mult * iqr)
    upper_thresh = q3 + (iqr_mult * iqr)
    if which == "upper":
        return upper_thresh
    elif which == "lower":
        return lower_thresh
    else:
        return lower_thresh, upper_thresh


def modified_zthresh(vec, thresh=2):
    const = 0.6745
    vec = np.asanyarray(vec)
    med = np.median(vec)
    abs_dev = np.abs(vec - med)
    mad = np.median(abs_dev)

    return ((thresh * mad) / const) + med


def bootstrap_tukey_outlier(vec, samples=10000, **kws):
    """Return a sorted vector of bootstraped outlier thresholds."""
    # Ensure vec is a 1D array
    vec = vec.flatten()

    # Bootstrap outlier thresholds
    output = []
    for _ in range(samples):
        sample_vals = np.random.choice(vec, size=vec.size, replace=True)
        output.append(tukey_outlier(sample_vals, **kws))

    # Sort and return
    output = np.sort(output)
    return output


def bootstrap_modified_zthresh(vec, thresh=2, samples=10000):
    """Return a sorted vector of bootstraped outlier thresholds."""
    # Ensure vec is a 1D array
    vec = vec.flatten()

    # Bootstrap outlier thresholds
    output = []
    for _ in range(samples):
        sample_vals = np.random.choice(vec, size=vec.size, replace=True)
        output.append(modified_zthresh(sample_vals, thresh=thresh))

    # Sort and return
    output = np.sort(output)
    return output


In [ ]:
save_output = True
overwrite = True
ssheet_dir = "/Users/dschonhaut/box/projects/leads_fdg_eononad/data/ssheets"
upper_pctl = 95
zthresh = 2

# Import data
infile = op.join(ssheet_dir, "Pourc_voxels_contrneg.csv")
dat = pd.read_csv(infile)
rois = dat.columns[1:]

# Calculate outlier thresholds
output = []
for roi in rois:
    boot_vals_tukey1_5 = bootstrap_tukey_outlier(dat[roi].values, iqr_mult=1.5)
    boot_vals_tukey3 = bootstrap_tukey_outlier(dat[roi].values, iqr_mult=3)
    boot_vals_modz = bootstrap_modified_zthresh(dat[roi].values, thresh=zthresh)
    output.append(
        [
            roi,
            np.percentile(dat[roi].values, upper_pctl),
            tukey_outlier(dat[roi].values, iqr_mult=1.5),
            np.mean(boot_vals_tukey1_5),
            tukey_outlier(dat[roi].values, iqr_mult=3),
            np.mean(boot_vals_tukey3),
            modified_zthresh(dat[roi].values, zthresh),
            np.mean(boot_vals_modz),
        ]
    )
output = pd.DataFrame(
    output,
    columns=[
        "roi",
        "p95",
        "tukey_thresh_1.5",
        "tukey_thresh_bootstrapped_mean_1.5",
        "tukey_thresh_3",
        "tukey_thresh_bootstrapped_mean_3",
        "modz_thresh",
        "modz_thresh_bootstrapped_mean",
    ],
)

# Save output
if save_output:
    outfile = op.join(
        ssheet_dir,
        f"LEADS_FDG_ref-region_Wscore_thresholds_61abeta-neg-controls_{today()}.csv",
    )
    if overwrite or not op.exists(outfile):
        output.to_csv(outfile, index=False)
        print(f"Saved: {outfile}")


In [ ]:
# Create a facet grid showing a histogram of values for each roi in datm["roi"]
datm = pd.melt(
    dat, id_vars="ID", value_vars=rois, var_name="roi", value_name="value"
).reset_index(drop=True)
g = sns.FacetGrid(
    datm, col="roi", col_wrap=6, height=2.5, aspect=1.2, sharex=False, sharey=False
)
g = g.map(plt.hist, "value", lw=0.1, ec="k")
# g.set_titles("{col_name}", pad=2)
# g.set_xlabels("W score")
# g.set_ylabels("Count")
# g.set(xlim=(-5, 5), ylim=(0, 40))
# sns.despine(trim=True)
g.fig.tight_layout()
